In [ ]:
import random

from distributed_processing.client import Client
from distributed_processing.redis_connector import RedisConnector
from distributed_processing.serializers import JsonSerializer

In [3]:
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_DB = 0
NAMESPACE = "tasks"

In [ ]:
redis_connector = RedisConnector(
    redis_host=REDIS_HOST, redis_port=REDIS_PORT, redis_db=REDIS_DB, namespace=NAMESPACE
)

client = Client(JsonSerializer(), redis_connector, check_registry="cache")

Client with id: tasks:redis_client:1
Results queue: tasks:redis_client:1:responses


In [5]:
y = client.rpc_async("add", [1, 0])

In [6]:
y.get()

1

In [ ]:
client.check_registry = "always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [ ]:
client.check_registry = "never"

client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Error -32601 : The method does not exist/is not available.

 


In [ ]:
client.check_registry = "never"

client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

'Esto es un error'

In [ ]:
client.check_registry = "cache"

In [11]:
x = client.rpc_async("div", [1, 0])

In [12]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "C:\home\distributed_processing\python\distributed_processing\worker.py", line 152, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_22836\1866336200.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [13]:
client.rpc_sync("add", [1, 1])

2

In [ ]:
def f(x, y):
    return x + y


y = client.rpc_async_fn(f, [1, 2.0])

In [15]:
y.get()

3.0

In [16]:
client.rpc_sync_fn(f, [3.0, 2.0])

5.0

In [ ]:
fs = []
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(), random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

('div', [0.2860268136171815, 0.35700636392307095], {})
('mul', [0.6159728070970859, 0.3102479533420225], {})
('mul', [0.7147953073646237, 0.585067914847418], {})
('div', [0.20523872923812236, 0.008786633779077846], {})
('div', [0.06828375074851778, 0.6178123955010395], {})
('add', [0.9804850645862808, 0.35795147098943125], {})
('div', [0.5678593331659472, 0.792199194964221], {})
('div', [0.23821177539787342, 0.21831609127477414], {})
('add', [0.05428189170120312, 0.8508874855785306], {})
('div', [0.042743144619149076, 0.7865624457538398], {})


In [18]:
[f.get() for f in fs]

[0.8011812744010794,
 0.19110430271621132,
 0.41820380002253965,
 23.358061164085765,
 0.11052505784242214,
 1.338436535575712,
 0.7168138225532962,
 1.09113246763867,
 0.9051693772797337,
 0.054341705289761365]

In [19]:
tp

[('div', [0.2860268136171815, 0.35700636392307095], {}),
 ('mul', [0.6159728070970859, 0.3102479533420225], {}),
 ('mul', [0.7147953073646237, 0.585067914847418], {}),
 ('div', [0.20523872923812236, 0.008786633779077846], {}),
 ('div', [0.06828375074851778, 0.6178123955010395], {}),
 ('add', [0.9804850645862808, 0.35795147098943125], {}),
 ('div', [0.5678593331659472, 0.792199194964221], {}),
 ('div', [0.23821177539787342, 0.21831609127477414], {}),
 ('add', [0.05428189170120312, 0.8508874855785306], {}),
 ('div', [0.042743144619149076, 0.7865624457538398], {})]

In [20]:
fs = client.rpc_batch_async(tp)

In [21]:
[f.get() for f in fs]

[0.8011812744010794,
 0.19110430271621132,
 0.41820380002253965,
 23.358061164085765,
 0.11052505784242214,
 1.338436535575712,
 0.7168138225532962,
 1.09113246763867,
 0.9051693772797337,
 0.054341705289761365]

In [22]:
client.rpc_batch_sync(tp)

[0.8011812744010794,
 0.19110430271621132,
 0.41820380002253965,
 23.358061164085765,
 0.11052505784242214,
 1.338436535575712,
 0.7168138225532962,
 1.09113246763867,
 0.9051693772797337,
 0.054341705289761365]

In [ ]:
fs = []
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(), random.random()], {})
    print(t)
    tp.append(t)

('fake', [0.8713724582894152, 0.5111450867735158], {})
('div', [0.6572236159560393, 0.8517809392660718], {})
('add', [0.3966966671180614, 0.022362080937775475], {})
('add', [0.16809092888613986, 0.916222755931137], {})
('div', [0.020740611545411847, 0.29930727276783686], {})
('mul', [0.7601012177385686, 0.15362759477321153], {})
('mul', [0.9701519219151844, 0.2973602352509014], {})
('mul', [0.3942511848340503, 0.33211813491336717], {})
('add', [0.2995676455936074, 0.6560277754442416], {})
('add', [0.8812625552577051, 0.03293380699403714], {})


In [24]:
tp

[('fake', [0.8713724582894152, 0.5111450867735158], {}),
 ('div', [0.6572236159560393, 0.8517809392660718], {}),
 ('add', [0.3966966671180614, 0.022362080937775475], {}),
 ('add', [0.16809092888613986, 0.916222755931137], {}),
 ('div', [0.020740611545411847, 0.29930727276783686], {}),
 ('mul', [0.7601012177385686, 0.15362759477321153], {}),
 ('mul', [0.9701519219151844, 0.2973602352509014], {}),
 ('mul', [0.3942511848340503, 0.33211813491336717], {}),
 ('add', [0.2995676455936074, 0.6560277754442416], {}),
 ('add', [0.8812625552577051, 0.03293380699403714], {})]

In [ ]:
client.check_registry = "never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

In [26]:
[f.safe_get() for f in fs]

[None,
 0.7715876062245878,
 0.41905874805583687,
 1.084313684817277,
 0.0692953811433098,
 0.11677252186536545,
 0.2884846037298133,
 0.13093796819446998,
 0.955595421037849,
 0.9141963622517423]

In [ ]:
client.check_registry = "cache"

In [28]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [ ]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [ ]:
client.check_registry = "never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

In [31]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [32]:
y = client.rpc_async("add", [1, 0])

In [33]:
y.get(5)

1

In [ ]:
def f(x, y):
    return x + y


client.check_registry = "never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 
